## Crawl and Scrape all info from the category: drugs

In [1]:
import socks
import socket
import requests
from bs4 import BeautifulSoup
import time
import re
import numpy as np
import pandas as pd

socks.set_default_proxy(socks.SOCKS5, "localhost", 9050) # one source said port 9150, Georgia's code used 9050
socket.socket = socks.socksocket

# These lines are copied from the Linkedin post.
def getaddrinfo(*args):
    return [(socket.AF_INET, socket.SOCK_STREAM, 6, '', (args[0], args[1]))]
socket.getaddrinfo = getaddrinfo

# ToRReZ market URL DEZE LINKE GEBRUIKEN OM DE COOCKIES TE PAKKEN
url = "http://yxuy5oau7nugw4kpb4lclrqdbixp3wvc4iuiad23ebyp2q3gx7rtrgqd.onion/items/category/drugs-and-chemicals?order=none&vendors=all&shipping_to=any&shipping_from=any&product_type=any&payments_method=any&page=1"


# Manually selected from the browser header after solving the captcha
cookies_copied = 'Cookie: ray_id=9bc0af157bce9cafa0a2b005a3ddc26d141c4aa7308e2e496c1cfbb089a07031; hr4ujvby8ds459og4kzcpbzwjdj_session=eyJpdiI6IjBIYVYzcWhrcng0dXFQazNaYlJxV2c9PSIsInZhbHVlIjoiWm9WOXJ6N3l0bUkxaXVyUGNcLzBpSXFrQ3VCMEFkNDRaYUplS2d6OVd5a09TOFdENm96UzFNdFoxdE9FYkFxa1wvTWVldkRqMktVT29JNWxIQUlaRmNpWnpPc0RWTXFUSEZJTnd6TFU5N2Nja1dVSXUxUXdTZTRVZ2V4RnJ2UW1pbSIsIm1hYyI6IjIzZjFlN2Y0MWE4MjRjMzIyN2UwZDQxMGRhNmFlOGJmODFjZmEwMDUzMTFjYTJlYTdjM2E3YmJiMjRlODM5NWUifQ%3D%3D; XSRF-TOKEN=eyJpdiI6ImxWelVNWUFiK3pJTE9iUitvRHRySUE9PSIsInZhbHVlIjoiaXhoMXloNVNFbU9pN3NrTWNQb0xKbkJibWtBMnNxdHFoYWJ4MkRhcmRWMjI2RUJkbWpwMGV0cFNLSGUxcDdVVENRbUdCSXBENm13SGlsUmRBTEJ0WloxaU1adnZlakxVdnVtSHo0WTR3cHV3N0Z1ZTBzZk02SFRxYlV5SkRBQ2QiLCJtYWMiOiJmMTVlMzMyZTkxNWZkNzJhOTNhNWQ3YzlhNjA1YzU2NmE1NmQzNDFlYzMzMDE3NmY3ZjkyMjdiYTc5YmUxNzlkIn0%3D'


# Takes the relevant info of the copied cookies and puts it in the correct format automatically.
def convert_pasted_cookies_to_usable_format(pasted_cookies):
    all_cookies_str = pasted_cookies.split(':')[1] # to ignore the prefix "Cookie: ")
    individual_cookies = all_cookies_str.split(';')
    cookies_set = {}
    for cookie in individual_cookies:
        cookies_set.update({cookie.strip().split('=')[0]: cookie.strip().split('=')[1]})
    return cookies_set


s = requests.Session()
s.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0'
})

proxies = {
    'http': 'socks5h://localhost:9050',
    'https': 'socks5h://localhost:9050'
}

set_of_cookies = convert_pasted_cookies_to_usable_format(cookies_copied)

# Get the page category drugs
web_page = s.get(url, cookies=set_of_cookies, proxies=proxies)
soup = BeautifulSoup(web_page.content, 'html.parser')



# create empty lists
vendor = []    
vendor_offers = []
all_titles = []
all_quantities = []
all_prices = []
category = []
category_level_1 = []
category_level_2 = []
category_level_3 = []
ship_from = []
ship_to =[]



# crawl and scrape through pages
pages = []
for a in soup.find_all('a',attrs={"class" : "page-link"}):
    pages.append(a.text)
last_page = pages[-2]

for page in range(1,int(last_page)+1): #int(last_page)+1

    # NIET AANPASSEN
    url = 'http://yxuy5oau7nugw4kpb4lclrqdbixp3wvc4iuiad23ebyp2q3gx7rtrgqd.onion/items/category/drugs-and-chemicals?order=none&vendors=all&shipping_to=any&shipping_from=any&product_type=any&payments_method=any&page=' + str(page)

    web_page = s.get(url, cookies=set_of_cookies, proxies=proxies)
    soup = BeautifulSoup(web_page.content, 'html.parser')


    for a in soup.find_all('a', href=True, class_='title'):
        all_titles.append(a.text.strip())
        if "gram" in a.text.strip().lower():
            #print(a.text.strip())
            if "." in a.text.strip().lower().split('gram')[0]:
                quantity = (re.findall("\d+\.\d+", a.text.strip().lower().split('gram')[0]))[0]
            else:
                quantity = (re.findall("\d+", a.text.strip().lower().split('gram')[0]))[0]
        else:
            #print("No quantity found in: " + a.text.strip())
            quantity = 0

        all_quantities.append(quantity)

    for price in soup.find_all('span', class_='Price'):
        all_prices.append(re.findall("\d+\.\d+", price.text.strip())[0])
        
    for a in soup.find_all('div',attrs={"class" : "w-100 mt-3 small"}):
        if len(a.find_all('a'))==4:
            category_level_1.append(a.find_all('a')[1].text.strip())
            category_level_2.append(a.find_all('a')[2].text.strip())
            category_level_3.append(a.find_all('a')[3].text.strip())
        elif len(a.find_all('a'))==3:
            category_level_1.append(a.find_all('a')[1].text.strip())
            category_level_2.append(a.find_all('a')[2].text.strip())
            category_level_3.append(np.nan)
        elif len(a.find_all('a'))==2:
            category_level_1.append(a.find_all('a')[1].text.strip())
            category_level_2.append(np.nan)
            category_level_3.append(np.nan)
        last_item = a.find_all('a')[-1]
        category.append(last_item.text.strip())

           
    for tag_elm in soup.find_all('div', class_='shipping'):
        if tag_elm.find('span', class_='shippingFrom') != None:
            ship_from.append(tag_elm.find('span', class_='shippingFrom').text.strip())
        else:
            ship_from.append(np.nan)

    for tag_elm in soup.find_all('div', class_='shipping'):
        if tag_elm.find('span', class_='shippingTo') != None:
            ship_to.append(tag_elm.find('span', class_='shippingTo').text.strip())
        else:
            ship_to.append(np.nan)

    for a in soup.find_all('a', class_='font-weight-bold'):
        vendor.append(a.text.strip())


            
            
results = (list(zip(vendor, category_level_1, category_level_2, category_level_3, category, all_titles, all_quantities, all_prices, ship_from, ship_to)))



df = pd.DataFrame(results,columns=['vendor', 'category_level_1', 'category_level_2', 'category_level_3', 'highest_category', 'product','quantity','price in $', 'shipping_from', 'shipping_to'])    


In [2]:
df

,vendor,category_level_1,category_level_2,category_level_3,highest_category,product,quantity,price in $,shipping_from,shipping_to
0,FENDI,Drugs and Chemicals,NaN,NaN,Drugs and Chemicals,1gr of Purple Guava - Amsterdam Import - THC 24%,0,19.53,United Kingdom,United Kingdom
1,Dimma,Drugs and Chemicals,Opiates,Oxycodone,Oxycodone,Oxycodone x1 M30,0,13.95,United Kingdom,Worldwide
2,NarcoticsWorldwide,Drugs and Chemicals,Cannabis & Hash,Buds & Flowers,Buds & Flowers,10 Grams // Amnesia Haze Weed Indoor \\,10,144.41,Germany,Worldwide
3,DutchDrugz,Drugs and Chemicals,Psychedelics,2C-B,2C-B,5 2CB Pills || Dutch Quality 2C-B pills || 13 mg,0,30.09,Netherlands,Worldwide
4,WhiteDoc,Drugs and Chemicals,Stimulants,Cocaine,Cocaine,7G THE FAMOUS WHITEOUT COCAINE BATCH 92% PURITY,0,390.00,United States,United States
5,ShinyFlakez,Drugs and Chemicals,Ecstasy,MDMA,MDMA,5 Gr Top Quality MDMA,0,36.10,Netherlands,Worldwide
6,AmsterdamNL,Drugs and Chemicals,Psychedelics,2C-B,2C-B,🇩🇪 50x ★★ 2CB GOLDEN OREO 30MG 2CB ★★ HARD PRESS,0,102.29,Germany,Europe
7,shroomer,Drugs and Chemicals,Psychedelics,Mushrooms,Mushrooms,4g Magic Mushrooms cubensis shrooms Zauberpilze,0,43.32,Germany,Europe
8,RXChemist,Drugs and Chemicals,Prescriptions Drugs,NaN,Prescriptions Drugs,50 tabs LADYGRA 100 / Sildenafil woman 25 usd,0,25.00,United Kingdom,United Kingdom
9,RoyalMailer,Drugs and Chemicals,Ecstasy,Pills,Pills,5x Orange Tesla XTC 250mg IMPORT - UK,0,34.87,United Kingdom,United Kingdom


## Crawl and Scrape all vendor information

In [3]:
unique_vendors = list(set(vendor))

vendor_page = []

for vendor in unique_vendors:
    vendor_page.append("http://yxuy5oau7nugw4kpb4lclrqdbixp3wvc4iuiad23ebyp2q3gx7rtrgqd.onion/profile/" + str(vendor))

since = [] 
transactions = []
feedback_total = []
feedback_positive = []
feedback_negative = []
disputes_total =[]
disputes_won = []
disputes_lost = []
finalize_early = []
rank = []
verification = []

all_variables = [since, transactions, feedback_total, feedback_positive, 
                 feedback_negative, disputes_total, disputes_won, disputes_lost, finalize_early]
all_text = ['On ToRReZ Market Since:', 'Total Amount Of Transactions', 'Total Feedback Received', 
            'Positive Feedback Received Ratio', 'Negative Feedback Received Ratio', 'Disputes Total:', 
            'Disputes Won:', 'Disputes Lost:', 'Finalize Early']    
    
for page in vendor_page:
    web_page = s.get(page, cookies=set_of_cookies, proxies=proxies)
    soup = BeautifulSoup(web_page.content, 'html.parser')
    
    for numb in range(0,len(all_variables)):
        for i in soup.find_all('table', class_='table table-borderless table-sm mt-1 table-fixed'):
            value = soup.find('th', text=all_text[numb])
            all_variables[numb].append(value.find_next_sibling('td').text.strip())

    for span in soup.find_all('span', class_='badge badge-success question'):
        rank.append(span.text.strip())
        
    for tag_elm in soup.find_all('div', class_='col singleItemDetails mt-2'):
        if tag_elm.find('a', class_='badge badge-dark') != None:
            verification.append(tag_elm.find('a', class_='badge badge-dark').text.strip())
        else:
            verification.append(np.nan)


In [4]:
results_2 = (list(zip(unique_vendors, rank, verification, since, transactions, feedback_total, feedback_positive, feedback_negative, 
                      disputes_total, disputes_won, disputes_lost, finalize_early)))



df_2 = pd.DataFrame(results_2,columns=['vendor', 'rank', 'verifcation', 'since', 'transactions', 'feedback_total', 
                                   'feedback_positive', 'feedback_negative','disputes_total',
                                   'disputes_won', 'disputes_lost', 'finalize_early'])    


In [5]:
df_2

,vendor,rank,verifcation,since,transactions,feedback_total,feedback_positive,feedback_negative,disputes_total,disputes_won,disputes_lost,finalize_early
0,xanking,Rank 0,NaN,"Jan 24, 2021",2,0,0% (0),0% (0),0,0% (0),0% (0),Not Available
1,AmsterdamNL,Rank 1,Verification Level 5,"Jan 21, 2021",135,53,100% (53),0% (0),1,0% (0),0% (0),Not Available
2,bestgroup,Rank 1,Verification Level 7,"Mar 20, 2020",52,11,54.55% (6),27.27% (3),1,0% (0),0% (0),Not Available
3,RoyalMailer,Rank 4,Verification Level 4,"Aug 11, 2020",469,177,82.49% (146),11.86% (21),29,13.79% (4),51.72% (15),Not Available
4,ShinyFlakez,Rank 3,Verification Level 4,"Nov 7, 2020",353,99,76.77% (76),15.15% (15),41,4.88% (2),36.59% (15),Not Available
5,WhiteDoc,Rank 0,Verification Level 2,"Jan 20, 2021",1,1,100% (1),0% (0),0,0% (0),0% (0),Not Available
6,RXChemist,Rank 2,Verification Level 3,"Aug 28, 2020",246,86,86.05% (74),9.30% (8),16,18.75% (3),12.50% (2),Available (can require)
7,FENDI,Rank 0,Verification Level 1,"Feb 8, 2021",6,2,100% (2),0% (0),0,0% (0),0% (0),Not Available
8,Pygmalion,Rank 4,Verification Level 8,"Oct 6, 2020",516,159,86.16% (137),8.18% (13),27,7.41% (2),0% (0),Available (can require)
9,shroomer,Rank 1,Verification Level 1,"Dec 2, 2020",61,27,100% (27),0% (0),2,0% (0),0% (0),Not Available
